In [127]:
import pandas as pd
import bs4 as bs
import re
import requests
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm

## Original Search Results

In [47]:
# import list of search strings
search_list = pd.read_csv('./sample_list.csv', skiprows=2)

In [93]:
# change spaces to %20 as prescribed in the new search url and store on list
query_list = search_list['Query'].str.replace(' ', '%20')
query_list = list(set(query_list))

In [94]:
# define query url
query_url = 'https://forum.cirruspilots.org/search?q='

In [138]:
# define site login function
def site_login():
    '''
    This is a helper function that logins to cirruspilots.org
    '''
    driver.get('https://passport.cirruspilots.org/')
    driver.find_element_by_id('dnn_ctr_Login_Login_DNN_txtUsername').send_keys('jloretizo')
    driver.find_element_by_id('dnn_ctr_Login_Login_DNN_txtPassword').send_keys('cth4h#@EoY*a')
    driver.find_element_by_id('login_btn').click()

In [115]:
# define the driver and execute login 
driver = webdriver.Safari()
site_login()

In [139]:
# get the results set
results_set = {}
for i in tqdm(query_list):
    driver.get(query_url + i)
    soup = bs.BeautifulSoup(driver.page_source, 'html.parser')
    links_list = []
    for j in soup.find_all(class_='fps-result'):
        href = j.find(class_='search-link')['href']
        links_list.append(href)
    results_set[i] = links_list[:10]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [180]:
colnames=[str(x) for x in list(range(0,10))]

In [190]:
rank_list = pd.DataFrame.from_dict(results_set, orient='index', columns=colnames)

In [209]:
rank_list = rank_list.reset_index()

In [222]:
rank_list.head()

,index,0,1,2,3,4,5,6,7,8,9
0,brakes%20during%20taxi,/t/cirrus-sr22-brake-problems/41040/5,/t/brake-failure/38890,/t/leg-amputation/77460/4,/t/sr22-short-field-landing/57810/10,/t/first-flight-in-n119cd/36694/5,/t/mexico-trip/41088/6,/t/operating-costs/41132/10,/t/sr20-characteristics/40657/9,/t/p-static-issue/41068/7,/t/sr22-down-in-az-3-fatalities/41201/19
1,capture%20glide%20slope%20above,/t/gns430-vloc-operation/37559/3,/t/ils-and-glide-slope-tracking/39883/2,/t/re-bad-example-jerry-greetings-from-the-new...,None,None,None,None,None,None,None
2,find%20flight%20instructor,/t/looking-for-a-flight-instructor-in-mn/64840/3,/t/is-becoming-a-certified-flight-instructor-w...,/t/flight-schools-with-sr20-in-ca/39945,/t/additional-information-pilot-proficiency-pr...,/t/atp-vs-american-flyers-vs-others/89145,/t/looking-for-dedicated-intesnsive-sr22-ira-t...,/t/looking-for-csip-instructor-in-the-ksat-are...,/t/value-of-experience-fun-in-instruction/39863,/t/get-cockpit-weather-now-for-an-affordable-p...,/t/lessons/38823
3,oxygen%20altitude,/t/sr22-or-turbo/41277,/t/skylane-owner-seeking-feedback-on-sr22/3989...,/t/sr22-real-life-fuel-consumption/39903,/t/turbo-worth-it/82660,/t/ozone-concentration-limiting/89149/4,/t/openmic-oxygen-mask-cyber-monday-clearance-...,/t/introducing-the-precise-flight-boom-cannula...,/t/portable-oxygen-systems/43670/2,/t/supplemental-oxygen/37483/2,/t/oxygen-use-of-aerox-portable-or-built-in-ki...
4,stall%20under%20hood,/t/sr20-g2-stec55x-autopilot/67277/20,None,None,None,None,None,None,None,None,None


In [223]:
pd.melt(rank_list, id_vars=colnames, value_name='index')

,0,1,2,3,4,5,6,7,8,9,variable,index
0,/t/cirrus-sr22-brake-problems/41040/5,/t/brake-failure/38890,/t/leg-amputation/77460/4,/t/sr22-short-field-landing/57810/10,/t/first-flight-in-n119cd/36694/5,/t/mexico-trip/41088/6,/t/operating-costs/41132/10,/t/sr20-characteristics/40657/9,/t/p-static-issue/41068/7,/t/sr22-down-in-az-3-fatalities/41201/19,index,brakes%20during%20taxi
1,/t/gns430-vloc-operation/37559/3,/t/ils-and-glide-slope-tracking/39883/2,/t/re-bad-example-jerry-greetings-from-the-new...,None,None,None,None,None,None,None,index,capture%20glide%20slope%20above
2,/t/looking-for-a-flight-instructor-in-mn/64840/3,/t/is-becoming-a-certified-flight-instructor-w...,/t/flight-schools-with-sr20-in-ca/39945,/t/additional-information-pilot-proficiency-pr...,/t/atp-vs-american-flyers-vs-others/89145,/t/looking-for-dedicated-intesnsive-sr22-ira-t...,/t/looking-for-csip-instructor-in-the-ksat-are...,/t/value-of-experience-fun-in-instruction/39863,/t/get-cockpit-weather-now-for-an-affordable-p...,/t/lessons/38823,index,find%20flight%20instructor
3,/t/sr22-or-turbo/41277,/t/skylane-owner-seeking-feedback-on-sr22/3989...,/t/sr22-real-life-fuel-consumption/39903,/t/turbo-worth-it/82660,/t/ozone-concentration-limiting/89149/4,/t/openmic-oxygen-mask-cyber-monday-clearance-...,/t/introducing-the-precise-flight-boom-cannula...,/t/portable-oxygen-systems/43670/2,/t/supplemental-oxygen/37483/2,/t/oxygen-use-of-aerox-portable-or-built-in-ki...,index,oxygen%20altitude
4,/t/sr20-g2-stec55x-autopilot/67277/20,None,None,None,None,None,None,None,None,None,index,stall%20under%20hood
5,/t/do-you-still-call-for-a-briefing/80020/7,/t/ghastly-reminder/40075/4,/t/forward-vis-in-ice/40218/8,None,None,None,None,None,None,None,index,file%20nasa%20report
6,/t/an-accident-story-well-worth-reading-fly-th...,None,None,None,None,None,None,None,None,None,index,loose%20fairing%20noise
7,/t/flying-kids/39454/13,/t/seat-harness-problem-for-tall-pilots/76982/27,/t/our-plane-coming-to-northern-virginia/42742/16,/t/problems-with-my-new-cirrus/40940,/t/lines-that-work-on-spouses/36762/3,None,None,None,None,None,index,loose%20seat%20belt
8,/t/sr-20-landing-power-settings/40724,/t/shorts-and-softs/37089/17,/t/fs-special-for-copa-the-polar-icebox-summer...,/t/landing-tips/40436,/t/dead-stick-landing-techique/36963,/t/landing-a-sr22-vs-182/39294/4,/t/sr20-landing-light-stc/39498/3,/t/do-i-chute-a-landing-or-not/40042/2,/t/took-the-first-step/67586/12,/t/flight-school-training-in-a-cirrus-vs-tradi...,index,power%20off%20landing
9,None,None,None,None,None,None,None,None,None,None,index,music%20volume%20low


In [219]:
pd.DataFrame(rank_list.stack()).reset_index().pivot(columns='level_0', index='level_1')

0  \
level_0                                         0   
level_1                                             
0           /t/cirrus-sr22-brake-problems/41040/5   
1                          /t/brake-failure/38890   
2                       /t/leg-amputation/77460/4   
3            /t/sr22-short-field-landing/57810/10   
4               /t/first-flight-in-n119cd/36694/5   
5                          /t/mexico-trip/41088/6   
6                     /t/operating-costs/41132/10   
7                 /t/sr20-characteristics/40657/9   
8                       /t/p-static-issue/41068/7   
9        /t/sr22-down-in-az-3-fatalities/41201/19   
index                      brakes%20during%20taxi   

                                                            \
level_0                                                  1   
level_1                                                      
0                         /t/gns430-vloc-operation/37559/3   
1                  /t/ils-and-glide-slope-tracking/39883/2   
2        /t/re-bad-example-jerry-greetings-from-the-new...   
3                                                      NaN   
4                                                      NaN   
5                                                      NaN   
6                                                      NaN   
7                                                      NaN   
8                                                      NaN   
9                                                      NaN   
index                      capture%20glide%20slope%20above   

                                                            \
level_0                                                  2   
level_1                                                      
0         /t/looking-for-a-flight-instructor-in-mn/64840/3   
1        /t/is-becoming-a-certified-flight-instructor-w...   
2                  /t/flight-schools-with-sr20-in-ca/39945   
3        /t/additional-information-pilot-proficiency-pr...   
4                /t/atp-vs-american-flyers-vs-others/89145   
5        /t/looking-for-dedicated-intesnsive-sr22-ira-t...   
6        /t/looking-for-csip-instructor-in-the-ksat-are...   
7          /t/value-of-experience-fun-in-instruction/39863   
8        /t/get-cockpit-weather-now-for-an-affordable-p...   
9                                         /t/lessons/38823   
index                           find%20flight%20instructor   

                                                            \
level_0                                                  3   
level_1                                                      
0                                   /t/sr22-or-turbo/41277   
1        /t/skylane-owner-seeking-feedback-on-sr22/3989...   
2                 /t/sr22-real-life-fuel-consumption/39903   
3                                  /t/turbo-worth-it/82660   
4                  /t/ozone-concentration-limiting/89149/4   
5        /t/openmic-oxygen-mask-cyber-monday-clearance-...   
6        /t/introducing-the-precise-flight-boom-cannula...   
7                       /t/portable-oxygen-systems/43670/2   
8                           /t/supplemental-oxygen/37483/2   
9        /t/oxygen-use-of-aerox-portable-or-built-in-ki...   
index                                    oxygen%20altitude   

                                                \
level_0                                      4   
level_1                                          
0        /t/sr20-g2-stec55x-autopilot/67277/20   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                          NaN   
8                                          NaN   
9                                          NaN   
index                     stall%20under%20hood  

## New Search Results

In [4]:
# open the search html template
with open('search.html', 'r') as open_file:
    soup = bs.BeautifulSoup(open_file, 'html.parser')

In [37]:
# find all content item classes
for i in soup.find_all(class_='content-item'):
    # change the value of title inside the a tag
    i.find(class_='abbreviated-post').find('h4').find('a')['title'] = 'New Title'
    # change the href for link
    i.find(class_='abbreviated-post').find('h4').find('a')['href'] = 'New Title'
    # change text value
    i.find(class_='abbreviated-post').find('h4').find('a').string.replace_with("No longer bold")
    

In [38]:
# save this new html as test panel
with open("output1.html", "w") as file:
    file.write(str(soup))

In [41]:
soup.find_all(class_='content-item')[0].find('h4')

<h4 class="post-name"><a class="internal-link view-post" href="https://www.cirruspilots.org/copa/safety_programs/f/349/t/159914.aspx" title="New Title">No longer bold</a></h4>